In [3]:
import numpy as np
import pandas as pd
from scipy import misc

import os
import shutil
from tqdm import tqdm 

# Delete rubbish

In [4]:
# the folder from 256_ObjectCategories.tar file
dir_train = '/home/dongsung/Downloads/caltech-256-image-dataset/256_ObjectCategories'

In [5]:
# remove non-images
os.remove(os.path.join(dir_train, '198.spider/RENAME2'))
shutil.rmtree(os.path.join(dir_train, '056.dog/greg'))
# we don't need the class with noise
#shutil.rmtree(os.path.join(dir_train, '257.clutter'))

# Collect metadata

In [6]:
subdirs = list(os.walk(dir_train))[1:]

# collect train metadata
train_metadata = []

for dir_path, _, files in tqdm(subdirs):
    
    dir_name = dir_path.split('/')[-1]
    
    for file_name in files:
        if not file_name.startswith('.'):
            # read image
            temp = misc.imread(os.path.join(dir_path, file_name)) 
            # collect image metadata
            image_metadata = []
            image_metadata.extend([dir_name, file_name])
            image_metadata.extend( 
                list(temp.shape) if len(temp.shape) == 3 
                else [temp.shape[0], temp.shape[1], 1]
            )
            image_metadata.extend([temp.nbytes, temp.dtype])
            # append image metadata to list
            train_metadata.append(image_metadata)

  0%|          | 0/257 [00:00<?, ?it/s]/home/dongsung/tensorflow_virtualenv/lib/python3.5/site-packages/ipykernel_launcher.py:13: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  del sys.path[0]
100%|██████████| 257/257 [07:52<00:00,  1.89s/it]


# Explore metadata

In [7]:
M = pd.DataFrame(train_metadata)
M.columns = ['directory', 'img_name', 'height', 'width', 'channels', 'byte_size', 'bit_depth']

M['category_name'] = M.directory.apply(lambda x: x.split('.')[-1].lower())
M['img_extension'] = M.img_name.apply(lambda x: x.split('.')[-1])
M['category_number'] = M.directory.apply(lambda x: int(x.split('.')[0]))

# remove '101' from some category names
M.category_name = M.category_name.apply(lambda x: x[:-4] if '101' in x else x)

In [8]:
# number of grayscale images
(M.channels != 3).sum()

422

In [9]:
M.img_extension.unique()

array(['jpg'], dtype=object)

In [10]:
M.bit_depth.unique()

array([dtype('uint8')], dtype=object)

In [11]:
# number of categories
M.category_name.nunique()

257

# Create decoder

In [12]:
# class number -> class name
decode = {n: i for i, n in M.groupby('category_name').category_number.first().iteritems()}

In [13]:
np.save('decode.npy', decode)

# Split data

In [14]:
# 20 images per class
V = M.groupby('category_name', group_keys=False).apply(lambda x: x.sample(n=20, replace=False))
V.sort_index(inplace=True)
M.drop(V.index, axis=0, inplace=True)

In [15]:
# train data
len(M)

25467

In [16]:
# validation data
len(V)

5140

# Save split

In [17]:
M.to_csv('train_metadata.csv', index=False)

In [18]:
V.to_csv('val_metadata.csv', index=False)